In [17]:
%config IPCompleter.greedy=True

In [18]:
import numpy as np
import matplotlib.pyplot as plt
import praktikum as p
import pandas as pd

In [19]:
data = pd.read_excel(r'C:\Users\balaz\Desktop\MFF_UK\2. rocník ZS\praktikum\mer5\mer5.xlsx')

In [20]:
volt_m_a = np.array(data.iloc[2:11, :2])
volt_m = {}
for key, value in volt_m_a:
    volt_m[key] = value*1000

s_volt_m = {}
for key, value in volt_m_a:
    if value*1000 < 6000:
        s_volt_m[key] = value*1000*0.9/100+1
    elif value*1000 < 60_000:
        s_volt_m[key] = value*1000*0.9/100+10
    elif value*1000 < 600_000:
        s_volt_m[key] = value*1000*0.9/100+100

In [21]:
amp_m_a = np.array(data.iloc[2:8,2:4])
amp_m = {}
for key, value in amp_m_a:
    amp_m[key] = value

s_amp_m = {}
for key, value in amp_m_a:
    if value < 600:
        s_amp_m[key] = value*0.9/100+0.1

In [22]:
from math import sqrt
def trida_pres(spec, trid=2):
    return spec*trid/sqrt(3)/100

In [23]:
v_spec = np.array(data.iloc[2:, 7])
v_val_a = np.array(data.iloc[2:, 8])
v_val_b = np.array(data.iloc[2:, 13])

a_spec = np.array(data.iloc[2:, 10])
a_val_a = np.array(data.iloc[2:, 11])
a_val_b = np.array(data.iloc[2:, 15])

U_a = np.array([spec*val/150 for spec, val in zip(v_spec, v_val_a)])
s_Ua = np.array([trida_pres(spec) for spec, val in zip(v_spec, v_val_a)])

I_a = np.array([spec*val/150 for spec, val in zip(a_spec, a_val_a)])
s_Ia = np.array([trida_pres(spec) for spec, val in zip(a_spec, a_val_a)])

U_b = np.array([spec*val/150 for spec, val in zip(v_spec, v_val_b)])
s_Ub = np.array([trida_pres(spec) for spec, val in zip(v_spec, v_val_b)])

I_b = np.array([spec*val/150 for spec, val in zip(a_spec, a_val_b)])
s_Ib = np.array([trida_pres(spec) for spec, val in zip(a_spec, a_val_b)])

In [24]:
def a_kor(U_a, I_a, spec_a):
    return U_a/(I_a/1000)-spec_a

def b_kor(U_b, I_b, spec_v):
    return 1/((I_b/1000)/U_b-1/spec_v)

In [25]:
%%time
s_amp_spec = [s_amp_m[a_spec[i]] for i in range(len(a_spec))]
s_volt_spec = [s_volt_m[v_spec[i]] for i in range(len(v_spec))]

amp_spec = [amp_m[a_spec[i]] for i in range(len(a_spec))]
volt_spec = [volt_m[v_spec[i]] for i in range(len(v_spec))]

R_a_kor = np.array([a_kor(U_a[i], I_a[i], amp_spec[i]) for i in range(len(U_a))])
s_R_a_kor = p.prenos_chyb_multi(a_kor, [s_Ua, s_Ia, s_amp_spec], [U_a, I_a, amp_spec])

R_b_kor = np.array([b_kor(U_b[i], I_b[i], volt_spec[i]) for i in range(len(U_b))])
s_R_b_kor = p.prenos_chyb_multi(b_kor, [s_Ub, s_Ib, s_volt_spec], [U_b, I_b, volt_spec])

CPU times: total: 0 ns
Wall time: 1.07 ms


In [26]:
def div(a, b):
    return a/(b/1000)
R_a = div(U_a, I_a)
s_Ra = p.prenos_chyb_multi(div, [s_Ua, s_Ia], [U_a, I_a])

R_b = div(U_b, I_b)
s_Rb = p.prenos_chyb_multi(div, [s_Ub, s_Ib], [U_b, I_b])

R = div(U_b, I_a)
s_R = p.prenos_chyb_multi(div, [s_Ub, s_Ia], [U_b, I_a])

In [27]:
table1 = p.default_table(pd.DataFrame({'$I_{max}$ [mA]': p.readable(amp_m.keys()),
                       '$R$ [$\Omega$]': p.readable(amp_m.values(), s_amp_m.values())}), 'table1', 'odpory analógového ampérmetra pre všetky rozsahy')
table2 = p.default_table(pd.DataFrame({'$U_{max}$ [V]': p.readable(volt_m.keys()),
                       '$R$ [$k\Omega$]': p.readable(np.array(list(volt_m.values()))/1000, np.array(list(s_volt_m.values()))/1000)}), "table2", "odpory analógového voltmetra pre všetky rozsahy")

In [28]:
table3 = p.default_table(pd.DataFrame({'$U_a$ [V]': p.readable(U_a, s_Ua),
                       '$I_a$ [mA]': p.readable(I_a, s_Ia),
                       '$U_b$ [V]': p.readable(U_b, s_Ub),
                       '$I_b$ [mA]': p.readable(I_b, s_Ib)}), 'table3', 'namerané hodnoty napätia a prúdu pre zapojenia $a$ a $b$ včetne chyb meracích prístrojov')

In [33]:
table4 = p.default_table(
    pd.DataFrame({
        '$R_a \enspace [\Omega]$ - bez korekcie': p.readable(R_a, s_Ra),
        '$R_{a_{kor}} \enspace [\Omega]$ - s korekciou': p.readable(R_a_kor, s_R_a_kor),
        '$R_b \enspace [\Omega]$ - bez korekcie': p.readable(R_b, s_Rb),
        '$R_{b_{kor}} \enspace [\Omega]$ - s korekciou': p.readable(R_b_kor, s_R_b_kor)
        #,'$R \enspace [\Omega]$': p.readable(R, s_R)
    }), 
    'table4',
    'vypočítané hodnoty odporov pre zapojenia $a$ a $b$ s korekciu a bez korekcie a vypočítané hodnoty miešane $R = \frac{U_b}{I_a}')

In [51]:
I_a, I_b, R_a, R_a_kor, R_b_kor, s_Ia, s_Ib, s_Ra, s_Rb, s_R_a_kor, s_R_b_kor, s_R = np.array(I_a), np.array(I_b), np.array(R_a), np.array(R_a_kor), np.array(R_b_kor), np.array(s_Ia), np.array(s_Ib), np.array(s_Ra), np.array(s_Rb), np.array(s_R_a_kor), np.array(s_R_b_kor), np.array(s_R)

aa = np.argsort(I_a)
ab = np.argsort(I_b)
slc = slice(6, None)

fig, ax = p.default_plot([I_a[aa][slc], I_a[aa][slc], I_b[ab][slc], I_b[ab][slc], I_a[aa][slc]], 
                         [R_a[aa][slc], R_a_kor[aa][slc], R_b[ab][slc], R_b_kor[ab][slc], R[aa][slc]], 'I [mA]', 'R [$\Omega$]',
                         legend = ['$R_a \enspace [\Omega]$ - bez korekcie',
                                   '$R_{a_{kor}} \enspace [\Omega]$ - s korekciou',
                                   '$R_b \enspace [\Omega]$ - bez korekcie',
                                   '$R_{b_{kor}} \enspace [\Omega]$ - s korekciou',
                                   '$R \enspace [\Omega]$'],
                         xerror = [s_Ia[aa][slc], s_Ia[aa][slc], s_Ib[ab][slc], s_Ib[ab][slc], s_Ia[aa][slc]],
                         yerror = [s_Ra[aa][slc], s_R_a_kor[aa][slc], s_Rb[ab][slc], s_R_b_kor[ab][slc], s_R[aa][slc]], marker = "-o")

In [40]:
a_spec = np.array(data.iloc[2:8, 20])
a_val = np.array(data.iloc[2:8, 21])

I = np.array([spec*val/150 for spec, val in zip(a_spec, a_val)])
s_I = np.array([trida_pres(spec) for spec, val in zip(a_spec, a_val_a)])

R_n = np.array(data.iloc[2:8, 23])
s_Rn = np.array([1]*len(R_n))

In [41]:
table5 = pd.DataFrame({
    'I [mA]': p.readable(I, s_I),
    '$R_n \enspace [\Omega]$': p.readable(R_n, s_Rn)
})
table5 = p.default_table(table5, 'table5', 'namerané hodnoty prúdu a odporu substitučnou metódou')
print(table5)

\begin{table}[H]
	\caption{namerané hodnoty prúdu a odporu substitučnou metódou}
	\label{table5}
	\begin{center}
		\begin{tabular}{|l|l|l|}
			\cline{1-3}
		 & I [mA] & $R_n \enspace [\Omega]$ \\
			\cline{1-3}
		0 & $ 1.20 \pm  0.02$ & $161 \pm 1$ \\
			\cline{1-3}
		1 & $ 2.36 \pm  0.03$ & $111 \pm 1$ \\
			\cline{1-3}
		2 & $ 6.05 \pm  0.09$ & $223 \pm 1$ \\
			\cline{1-3}
		3 & $ 12.0 \pm  0.2$ & $353 \pm 1$ \\
			\cline{1-3}
		4 & $ 24.0 \pm  0.3$ & $546 \pm 1$ \\
			\cline{1-3}
		5 & $ 30.0 \pm  0.9$ & $637 \pm 1$ \\
			\cline{1-3}
	\end{tabular}
	\end{center}
\end{table}


In [86]:
fig1, ax1 = p.default_plot([I_a[aa][1:], I_b[ab][1:], I], 
                         [R_a_kor[aa][1:], R_b_kor[ab][1:], R_n], 'I [mA]', 'R [$\Omega$]',
                         legend = ['$R_{a_{kor}} \enspace [\Omega]$ - s korekciou',
                                   '$R_{b_{kor}} \enspace [\Omega]$ - s korekciou',
                                   '$R_n \enspace [\Omega]$'],
                         xerror = [s_Ia[aa][1:], s_Ib[ab][1:], s_I],
                         yerror = [s_R_a_kor[aa][1:], s_R_b_kor[ab][1:], s_Rn], marker = "-o")

In [71]:
%%time
slc = (I_a>1)*(I_a<2.5)
a, b = p.lin_fit(I_a[slc], R_a_kor[slc], [s_Ia[slc], s_R_a_kor[slc]])
a, s_a = a
b, s_b = b

CPU times: total: 484 ms
Wall time: 2.88 s


In [77]:
def lin(y, a, b):
    return (y-b)/a
R0 = 91
s_R0 = R0*0.9/100+0.2
R00 = lin(R0, a, b)
s_R00 = p.prenos_chyb(lin, [s_R0, s_a, s_b], [R0, a, b])